Flores Lara Alberto
5BV1
Ingenieria en IA

Generacion del cuerpo compuesto de 6 libros

In [69]:
import re
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from transformers import BertModel, BertTokenizer
import torch
from rake_nltk import Rake
import spacy
from collections import Counter
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.nlp.stemmers import Stemmer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer

In [70]:
Titulos = ["Dracula.txt", "The Strange Case of Dr. Jekyll and Mr. Hyde.txt", "Metamorphosis.txt","Riders of the Purple Sage.txt","The Light of the Western Stars.txt", "The Man of the Forest.txt"]
# Obtenemos el texto previamente almacenado en un archivo txt 
cuerpo = []
for titulo in Titulos:
    with open(titulo, 'r', encoding='utf-8') as archivo:
        Texto=archivo.read()
    cuerpo.append(Texto)


Limpieza del texto

In [71]:
# Definimos las funciones de preprocesamiento de texto
def minusculas(texto):
    texto_minusculas = texto.lower()
    return texto_minusculas

def caracteresesp(texto):
    texto_sin_caracteres = re.sub(r'[^a-zA-Z0-9\s]', '', texto)
    return texto_sin_caracteres

def quitaresp(texto):
    Texto_sin_espacios = " ".join(texto.split())
    return Texto_sin_espacios

def stooopwords(tokens):
    stop_words = set(stopwords.words('english'))
    texto_sin_stopword = [w for w in tokens if w.lower() not in stop_words]
    return texto_sin_stopword

def tokenizar(texto):
    tokens = word_tokenize(texto)
    return tokens

# Procesamos el texto de cada libro
def aplicar_procesamiento(cuerpos):
    cuerpos_procesados = []
    for cuerpo in cuerpos:
        cuerpo_procesado = minusculas(cuerpo)
        cuerpo_procesado = caracteresesp(cuerpo_procesado)
        cuerpo_procesado = quitaresp(cuerpo_procesado)
        tokens = tokenizar(cuerpo_procesado)
        tokens = stooopwords(tokens)
        cuerpos_procesados.append(tokens)
    return cuerpos_procesados

cuerpos_procesados = aplicar_procesamiento(cuerpo)

# Convertimos tokens del preprocesamiento a texto 
cuerpos_procesados = [' '.join(tokens) for tokens in cuerpos_procesados]

TF - IDF

In [72]:
# Inicializamos el vectorizador TF-IDF
vectorizer = TfidfVectorizer() 
tfidf = vectorizer.fit_transform(cuerpos_procesados) 

# Obtenemos las palabras
feature_names = vectorizer.get_feature_names_out()

# Iteramos sobre los libros
for i, texto in enumerate(cuerpos_procesados):
    # Obtener el vector TF-IDF del libro actual
    tfidf_vector = tfidf[i]

    # Obtenemos los índices de las palabras con mayor valor TF-IDF
    top_indices = np.argsort(tfidf_vector.toarray()).flatten()[::-1][:5]

    # Obtenemos las palabras más representativas
    top_words = [feature_names[ind] for ind in top_indices]

    # Se imprimen las palabras más representativas del libro
    print(f"Las 5 palabras más representativas de '{Titulos[i]}':")
    print(top_words)
    print()

Las 5 palabras más representativas de 'Dracula.txt':
['van', 'helsing', 'said', 'lucy', 'could']

Las 5 palabras más representativas de 'The Strange Case of Dr. Jekyll and Mr. Hyde.txt':
['utterson', 'jekyll', 'hyde', 'mr', 'poole']

Las 5 palabras más representativas de 'Metamorphosis.txt':
['gregor', 'gregors', 'would', 'room', 'could']

Las 5 palabras más representativas de 'Riders of the Purple Sage.txt':
['venters', 'jane', 'lassiter', 'bess', 'withersteen']

Las 5 palabras más representativas de 'The Light of the Western Stars.txt':
['madeline', 'stewart', 'stillwell', 'cowboys', 'nels']

Las 5 palabras más representativas de 'The Man of the Forest.txt':
['helen', 'bo', 'dale', 'roy', 'beasley']



BERT - Transformers

In [73]:
# Cargar el modelo BERT pre-entrenado y el tokenizador
model = BertModel.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def obtener_top_palabras(texto, top_n=5):
    # Tokenizamos el texto
    tokens = tokenizer(texto, return_tensors="pt", truncation=True, padding=True)
    
    # Pasamos el input por BERT para obtener los embeddings
    with torch.no_grad():
        output = model(**tokens)
    word_embeddings = output.last_hidden_state.squeeze(0)
    
    # Usamos la media de las representaciones para obtener el embedding de la oración
    sentence_embedding = word_embeddings.mean(dim=0)

    # Se calcula la similitud entre las palabras y la oración
    similarities = torch.nn.functional.cosine_similarity(word_embeddings, sentence_embedding.unsqueeze(0), dim=1)

    # Ordenamos por similitud y obtener los índices de las palabras más similares
    sorted_indices = torch.argsort(similarities, descending=True).tolist()
    
    # Convertir tokens a palabras originales
    tokens_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'].squeeze(0).tolist())

    # Obtenemos las top_n palabras más representativas
    top_palabras = [tokens_text[idx] for idx in sorted_indices if tokens_text[idx] not in ["[CLS]", "[SEP]"]][:top_n]

    return top_palabras

# Se imprimen las palabras más representativas del libro
for i, texto in enumerate(cuerpos_procesados):
    top_palabras = obtener_top_palabras(texto)
    print(f"Las 5 palabras principales de '{Titulos[i]}':")
    print(top_palabras)
    print()

c:\Users\albsa\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Las 5 palabras principales de 'Dracula.txt':
['took', 'far', 'go', 'stopped', 'good']

Las 5 palabras principales de 'The Strange Case of Dr. Jekyll and Mr. Hyde.txt':
['see', 'spoke', 'friends', 'never', 'face']

Las 5 palabras principales de 'Metamorphosis.txt':
['back', 'touched', 'thought', 'would', 'back']

Las 5 palabras principales de 'Riders of the Purple Sage.txt':
['mountainous', 'thought', 'wondered', 'vast', 'clouds']

Las 5 palabras principales de 'The Light of the Western Stars.txt':
['looked', 'silent', 'low', 'window', 'could']

Las 5 palabras principales de 'The Man of the Forest.txt':
['lay', 'lay', 'change', 'dark', 'miles']



Rake - Nltk

In [74]:
# Inicializamos el objeto de Rake
rake_nltk_var = Rake()

# Iterar sobre cada libro en el cuerpo
for i, texto in enumerate(cuerpo):
    # Extraemos las palabras clave del texto actual
    rake_nltk_var.extract_keywords_from_text(texto)
    keyword_extracted = rake_nltk_var.get_ranked_phrases()
    
    # Seleccionamos las 5 palabras principales
    top_keywords = keyword_extracted[:5]
    
    # Se imprimen las palabras más representativas del libro
    print(f"Las 5 palabras principales de '{Titulos[i]}':")
    print(top_keywords)
    print()

Las 5 palabras principales de 'Dracula.txt':
['“ lenore ”:-- “ denn die todten reiten schnell ”-- (“', 'repeater apiece !” “ good !” said van helsing', '‘ _dos pou sto_ ,’ said archimedes', '“ bloofer lady .” chapter xiv mina harker ’', '“ dear sirs ,-- “ herewith please receive invoice']

Las 5 palabras principales de 'The Strange Case of Dr. Jekyll and Mr. Hyde.txt':
['differently sloped .” “ rather quaint ,” said utterson', 'exorbitant alarms .” “ sir ,” said', 'period exceeding three calendar months ,”', '“ double ” occurring perhaps six times', 'useful .” “ yes ,” returned mr']

Las 5 palabras principales de 'Metamorphosis.txt':
['trainees make enquiries — assuming enquiries', 'room several times without anything decisive happening', 'little legs along one side hung quivering', 'electric street lamps shone palely', 'concealed behind carefully carved furniture full']

Las 5 palabras principales de 'Riders of the Purple Sage.txt':
['familiar characteristics — masked feeling — strang

Spacy

In [75]:
# Cargamos el modelo de spaCy
nlp = spacy.load("en_core_web_sm")

# Agregamos el TextRank al pipeline de spaCy
nlp.add_pipe("textrank")

# Iteramos sobre cada libro en el cuerpo
for i, texto in enumerate(cuerpos_procesados):
    # procesar el texto con spaCy
    doc = nlp(texto)
    
    # Se imprimen las frases más representativas del libro
    print(f"Las 5 frases principales de '{Titulos[i]}':")
    for j, phrase in enumerate(doc._.phrases[:5]):
        print(f"{j + 1}. {phrase.text}")
    print()

Las 5 frases principales de 'Dracula.txt':
1. say way
2. jonathan saw told
3. dr van helsing gone room
4. sight seen man
5. coming night

Las 5 frases principales de 'The Strange Case of Dr. Jekyll and Mr. Hyde.txt':
1. mr utterson lawyer
2. mr hyde
3. hyde hyde indifferent jekyll
4. mr utterson hard man
5. mr utterson hand

Las 5 frases principales de 'Metamorphosis.txt':
1. gregor gregor
2. door gregors room sister
3. efforts gregors mother sister look gregor
4. long gregor gregor
5. gregor room

Las 5 frases principales de 'Riders of the Purple Sage.txt':
1. jane venters
2. lassiter jane venters
3. rider venters
4. venters rider
5. come venters

Las 5 frases principales de 'The Light of the Western Stars.txt':
1. madeline madeline
2. madeline stillwell men
3. madeline madeline arranged coat
4. man madeline hammond
5. gene stewart madeline

Las 5 frases principales de 'The Man of the Forest.txt':
1. helen dale
2. helen dales
3. bo helen
4. helen bo
5. seeing helen



Resumen de los primeros capitulos

In [76]:
# Función para cargar y extraer el primer capítulo del libro
def cargar_y_extraer_primer_capitulo(nombre_archivo):
    with open(nombre_archivo, 'r', encoding='utf-8') as archivo:
        texto = archivo.read()
        # Definimos una re para identificar los capitulos
        capitulos = re.split(r'\nCHAPTER \w+', texto)
        # Devuelve el primer capítulo
        return capitulos[1] if len(capitulos) > 1 else texto

# Seleccionamos dos libros de nuestro cuerpo
titulos = ["Dracula.txt", "The Man of the Forest.txt"]
textos_primeros_capitulos = [cargar_y_extraer_primer_capitulo(titulo) for titulo in titulos]

Frecuencias normalizadas

In [77]:
# Cargamos el modelo de lenguaje de SpaCy
nlp = spacy.load('en_core_web_sm')

def generar_resumen(texto, num_sentencias=10):
    doc = nlp(texto)
    # Filtramos las palabras vacías y puntuar las palabras no vacías
    palabras = [token.text.lower() for token in doc if not token.is_stop and not token.is_punct]
    palabra_frecuencia = Counter(palabras)
    max_frecuencia = max(palabra_frecuencia.values())
    
    # Normalizamos frecuencias
    for palabra in palabra_frecuencia:
        palabra_frecuencia[palabra] = palabra_frecuencia[palabra] / max_frecuencia
    
    # Puntuamos las oraciones
    sentence_scores = {}
    for sent in doc.sents:
        for palabra in sent:
            if palabra.text.lower() in palabra_frecuencia:
                if sent in sentence_scores:
                    sentence_scores[sent] += palabra_frecuencia[palabra.text.lower()]
                else:
                    sentence_scores[sent] = palabra_frecuencia[palabra.text.lower()]
    
    # Seleccionamos las oraciones con mayor puntuación
    resumen = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentencias]
    return ' '.join([sent.text for sent in resumen])

# Se imprime el resumen del primer capitulo del libro
for i, resumen in enumerate(textos_primeros_capitulos):
    print(f"Resumen del libro {i+1}:")
    resumen = generar_resumen(resumen)
    print(resumen)
    print("\n")

Resumen del libro 1:
One
by one several of the passengers offered me gifts, which they pressed
upon me with an earnestness which would take no denial; these were
certainly of an odd and varied kind, but each was given in simple good
faith, with a kindly word, and a blessing, and that strange mixture of
fear-meaning movements which I had seen outside the hotel at
Bistritz--the sign of the cross and the guard against the evil eye.
 Sometimes, as the road was cut through the pine woods
that seemed in the darkness to be closing down upon us, great masses of
greyness, which here and there bestrewed the trees, produced a
peculiarly weird and solemn effect, which carried on the thoughts and
grim fancies engendered earlier in the evening, when the falling sunset
threw into strange relief the ghost-like clouds which amongst the
Carpathians seem to wind ceaselessly through the valleys. Right
and left of us they towered, with the afternoon sun falling full upon
them and bringing out all the glori

Gensim

# Función para resumir el texto
def resumir_texto(texto, porcentaje=0.05):
    resumen = summarize(texto, ratio=porcentaje)
    if not resumen:
         resumen = "No se pudo generar un resumen con el porcentaje dado."
    return resumen

# Aplicar el resumen a cada texto
resumenes = [resumir_texto(texto) for texto in textos_primeros_capitulos]

# Imprimir los resúmenes
for i, resumen in enumerate(resumenes):
    print(f"Resumen de '{titulos[i]}':\n{resumen}\n")

TextRank

In [78]:
def resumir_texto(texto, num_sentencias=10):
    # Inicializamos todo lo necesario para el TextRank 
    parser = PlaintextParser.from_string(texto, Tokenizer("english"))
    stemmer = Stemmer("english")
    summarizer = TextRankSummarizer(stemmer)
    
    # Aplicamos el sumarizador al documento creado por el parser para obtener las 10 oraciones
    resumen = summarizer(parser.document, num_sentencias)
    
    # Convertimos las oraciones en un solo string, separadas por saltos de línea
    return "\n".join(str(oracion) for oracion in resumen)

resumenes = [resumir_texto(texto) for texto in textos_primeros_capitulos]

# Se imprime el resumen del primer capitulo del libro
for i, resumen in enumerate(resumenes):
    print(f"Resumen de '{titulos[i]}':\n{resumen}\n")


Resumen de 'Dracula.txt':
I find that the district he named is in the extreme east of the country, just on the borders of three states, Transylvania, Moldavia and Bukovina, in the midst of the Carpathian mountains; one of the wildest and least known portions of Europe.
In the population of Transylvania there are four distinct nationalities: Saxons in the South, and mixed with them the Wallachs, who are the descendants of the Dacians; Magyars in the West, and Szekelys in the East and North.
I shall never forget the last glimpse which I had of the inn-yard and its crowd of picturesque figures, all crossing themselves, as they stood round the wide archway, with its background of rich foliage of oleander and orange trees in green tubs clustered in the centre of the yard.
Right and left of us they towered, with the afternoon sun falling full upon them and bringing out all the glorious colours of this beautiful range, deep blue and purple in the shadows of the peaks, green and brown where gr

LSA

In [79]:
def resumir_texto(texto, num_sentencias=10):
    # Inicializamos todo lo necesario para el TextRank 
    parser = PlaintextParser.from_string(texto, Tokenizer("english"))
    stemmer = Stemmer("english")
    summarizer = LsaSummarizer(stemmer)
    
    # Aplicamos el sumarizador al documento creado por el parser para obtener las 10 oraciones
    resumen = summarizer(parser.document, num_sentencias)
    
    # Convertimos las oraciones en un solo string, separadas por saltos de línea
    return "\n".join(str(oracion) for oracion in resumen)

# Aplicar el resumen a cada texto
resumenes = [resumir_texto(texto) for texto in textos_primeros_capitulos]

# Se imprime el resumen del primer capitulo del libro
for i, resumen in enumerate(resumenes):
    print(f"Resumen de '{titulos[i]}':\n{resumen}\n")

Resumen de 'Dracula.txt':
I had for dinner, or rather supper, a chicken done up some way with red pepper, which was very good but thirsty.
There was a dog howling all night under my window, which may have had something to do with it; or it may have been the paprika, for I had to drink up all the water in my carafe, and was still thirsty.
It was on the dark side of twilight when we got to Bistritz, which is a very interesting old place.
Fifty years ago a series of great fires took place, which made terrible havoc on five separate occasions.
He and his wife, the old lady who had received me, looked at each other in a frightened sort of way.
Just before I was leaving, the old lady came up to my room and said in a very hysterical way:
I must say they were not cheering to me, for amongst them were “Ordog”--Satan, “pokol”--hell, “stregoica”--witch, “vrolok” and “vlkoslak”--both of which mean the same thing, one being Slovak and the other Servian for something that is either were-wolf or vamp